In [29]:
from tika import parser # pdf scraping

import os
import pandas as pd
import re
import sqlite3

In [34]:
pdf1 = parser.from_file('data_02_05/3011RL.pdf')
pdf2 = parser.from_file('data_02_05/3010RL4.pdf')
pdf3 = parser.from_file('data_02_05/2005JP3007RL.pdf')
pdf4 = parser.from_file('data_02_05/2007JP3021RL.pdf')

In [3]:
def get_year(competition):
    """
    Get year the competition was held
    :param competition: competition data scraped from the pdf [dict]
    :return: competition year [int]
    """

    competition_text = competition['content']
    year = int(re.search('20[0,1][0-9]', competition_text).group())

    return year


def get_jury(competition):
    """
    Get a list of juries at the competition. It is returned as list of tuples containing judge name and his nationality.
    :param competition: competition data scraped from the pdf [dict]
    :return: list of lists (jury_name, nationality) [list]
    """

    jury = []
    competition_text = competition['content']
    upper_boundary = '\n\nA '
    # Line below finds where to the jury list begins and '-2' is to remain the A character for iteration
    upper_trim = competition_text[competition_text.find(upper_boundary)+len(upper_boundary)-2 :]

    # I have to iterate with A-E characters, as in some cases there are unexpected blank lines between particular judges
    for letter in ['A', 'B', 'C', 'D', 'E']:
        upper_trim = upper_trim[upper_trim.find(letter+' '):]
        cut_line = upper_trim.split('\n')[0]
        nationality = cut_line[-3:]
        judge_name = re.sub(',', '', cut_line[2: -4])
        jury.append([judge_name, nationality])

    assert len(jury) == 5, f'Invalid number of judges! Got {len(jury)}'

    return jury


def note_to_float(score):
    '''
    Convert note to float. Some of them contain dots and some contain comas, so change it if needed.
    :param score: score [str]
    :return: score [float]
    '''

    return float(re.sub(',', '.', score))


def get_referenced_notes(score):
    '''
    Get difference between particular note and mean of notes of other judges
    :param score: list of notes [list]
    :return: list of differences [list]
    '''

    return [note - (sum(score)-note)/4 for note in score]


def get_competition_scores(competition):
    '''
    Extract scores from the competition. If jumper took part in both rounds, append his jumps as two separate list elements.
    :param competition: competition data scraped from the pdf [dict]
    :return: list of lists containing [jumper_name, jumper_country, note_A, note_B, note_C, note_D, note_E] [list]
    '''

    competition_text = competition['content']

    # Remove dots after place number, if they appear
    competition_text = re.sub('[.] ', ' ', competition_text)

    # Search for text fragments which match places pattern and split the text
    # The pattern is as follows: two new line characters, one or zero spaces, one or zero digits [1-5], one digit [0-9], a space or a dot
    competition_split = re.split(r'\n\n ?[1-5]?[0-9][. ]', competition_text)

    scores = []

    for place_fragment in competition_split[1:]:

        # Remove blank lines in list elements containing place (sometimes they appear, and sometimes do not) and split fragment into lines
        place_fragment = re.sub('\n\n', '\n', place_fragment).splitlines()
        # Get jumper name with no comas and country
        jumper_name = re.sub(', ', ' ', place_fragment[1])
        jumper_country = place_fragment[3].split()[0]
        # Get first round notes
        jumper_notes = [note_to_float(note) for note in place_fragment[4].split()[3:8]]
        jumper_referenced_notes = get_referenced_notes(jumper_notes)
        # Append first round notes
        scores.append([jumper_name, jumper_country]+jumper_notes+jumper_referenced_notes)

        # Check first character of first element of next line - if it is a digit, that means it also contains notes,
        # which means jumper took part in final round. Append score as new list element
        if len(place_fragment) > 5:
            if place_fragment[5].split()[0][0].isdigit():
                jumper_notes = [note_to_float(note) for note in place_fragment[5].split()[3:8]]
                jumper_referenced_notes = get_referenced_notes(jumper_notes)
                scores.append([jumper_name, jumper_country]+jumper_notes+jumper_referenced_notes)

    return scores

In [12]:
# Create DataFrame to store competition data - year and judges names
competition_data_columns = ['competition_id', 'year', 'referee_A', 'referee_B', 'referee_C', 'referee_D', 'referee_E']
competition_data_df = pd.DataFrame(columns=competition_data_columns)

# Create DataFrame to store all notes
scores_df_columns = ['competition_id', 'jumper_name', 'jumper_country', 'note_A', 'note_B', 'note_C', 'note_D', 'note_E']
scores_df = pd.DataFrame(columns=scores_df_columns)

# Create DataFrame to store jury countries
jury_data_columns = ['judge_name', 'judge_nationality']
jury_data_df = pd.DataFrame(columns=jury_data_columns)

# Fill DataFrame with competitions scores
data_path = 'data/'
competition_scores_columns = ['jumper_name', 'jumper_country', 'note_A', 'note_B', 'note_C', 'note_D', 'note_E', 'referenced_A', 'referenced_B', 'referenced_C', 'referenced_D', 'referenced_E']

for file_name in os.listdir(data_path):

    # Competition scores
    file_path = data_path + file_name
    print(f'Scanning {file_path}')
    pdf = parser.from_file(file_path)
    competition_scores_list = get_competition_scores(pdf)
    competition_scores_df = pd.DataFrame(competition_scores_list, columns=competition_scores_columns)
    competition_scores_df['competition_id'] = file_name
    scores_df = pd.concat([scores_df, competition_scores_df])
    print(f'Jumps performed: {len(competition_scores_list)}')

    # Competition data
    jury_names = [x[0] for x in get_jury(pdf)]
    competition_data_df = pd.concat([
        competition_data_df,
        pd.DataFrame([[file_name, get_year(pdf)] + jury_names], columns=competition_data_columns)
    ])

    # Jury data
    for jury in get_jury(pdf):
        if jury[0] not in jury_data_df['judge_name'].to_numpy():
            jury_data_df = pd.concat([
                jury_data_df,
                pd.DataFrame([jury], columns=jury_data_columns)
            ])

Scanning data/2005JP3007RL.pdf
Jumps performed: 80
Scanning data/2005JP3008RL.pdf
Jumps performed: 80
Scanning data/2005JP3012RL.pdf
Jumps performed: 69
Scanning data/2005JP3013RL.pdf
Jumps performed: 70
Scanning data/2005JP3019RL.pdf
Jumps performed: 80
Scanning data/2005JP3020RL.pdf
Jumps performed: 80
Scanning data/2005JP3025RL.pdf
Jumps performed: 80
Scanning data/2005JP3032RL.pdf
Jumps performed: 79
Scanning data/2005JP3033RL.pdf
Jumps performed: 80
Scanning data/2005JP3051RL.pdf
Jumps performed: 80
Scanning data/2005JP3052RL.pdf
Jumps performed: 80
Scanning data/2005JP3054RL.pdf
Jumps performed: 80
Scanning data/2005JP3055RL.pdf
Jumps performed: 90
Scanning data/2005JP3056RL.pdf
Jumps performed: 80
Scanning data/2005JP3078RL.pdf
Jumps performed: 80
Scanning data/2005JP3089RL.pdf
Jumps performed: 73
Scanning data/2005JP3102RL.pdf
Jumps performed: 78
Scanning data/2005JP3103RL.pdf
Jumps performed: 80
Scanning data/2005JP3108RL.pdf
Jumps performed: 78
Scanning data/2005JP3110RL.pdf


# Let's check the data

## Competition data
As we can see, there is no null values, seems like the referees and competition years have been filled nice

In [6]:
competition_data_df.head(10)

,competition_id,year,referee_A,referee_B,referee_C,referee_D,referee_E
0,2005JP3007RL.pdf,2005,Schnetzer Manfred,Langer Andreas,Bakken Knut A.,Braz Vladimir,Nordin Hans
0,2005JP3008RL.pdf,2005,Nordin Hans,Bras Vladimir,Hess Bernd,Bakken Knut A.,Wolf Stefan
0,2005JP3012RL.pdf,2005,Justin Toni,Kaekilehto Juhani,Krab Gerhard,Minemura Yutaka,Aschenbach Dietmar
0,2005JP3013RL.pdf,2005,Krab Gerhard,Aschenbach Dietmar,Justin Toni,Minemura Yutaka,Kaekilehto Juhani
0,2005JP3019RL.pdf,2004,Hoehnl Rudolf,Stahlhut Erik,Livland Nils,Pichler Johann,Bester Janez
0,2005JP3020RL.pdf,2004,Bester Janez,Hoehnl Rudolf,Pichler Johann,Livland Nils,Stahlhut Erik
0,2005JP3025RL.pdf,2004,Evja Jan Bjoern,Walter Kurt,Kallio Tapio,Dlugopolski Kazimierz,Busk Per
0,2005JP3032RL.pdf,2005,Szostak Tadeusz,Junnonen Tapio,Nakamura Mitsushi,Solbu Jarle,Stattmann Peter
0,2005JP3033RL.pdf,2005,Nakamura Mitsushi,Stattmann Peter,Szostak Tadeusz,Huopainen Juha,Bakken Knut A.
0,2005JP3051RL.pdf,2004,Bachmayer Johann,Dahl Anton,Borstnar Zvonimir,Huenefeld Mirko,Nordin Hans


In [7]:
competition_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88 entries, 0 to 0
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   competition_id  88 non-null     object
 1   year            88 non-null     object
 2   referee_A       88 non-null     object
 3   referee_B       88 non-null     object
 4   referee_C       88 non-null     object
 5   referee_D       88 non-null     object
 6   referee_E       88 non-null     object
dtypes: object(7)
memory usage: 5.5+ KB


## Jury data
Also seems nice

In [222]:
jury_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 138 entries, 0 to 0
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   judge_name         138 non-null    object
 1   judge_nationality  138 non-null    object
dtypes: object(2)
memory usage: 3.2+ KB


In [223]:
jury_data_df.head(10)

,judge_name,judge_nationality
0,Schnetzer Manfred,AUT
0,Langer Andreas,GER
0,Bakken Knut A.,NOR
0,Braz Vladimir,SLO
0,Nordin Hans,SWE
0,Bras Vladimir,SLO
0,Hess Bernd,GER
0,Wolf Stefan,AUT
0,Justin Toni,SLO
0,Kaekilehto Juhani,FIN


## Competition scores
Let's see it above and discuss it later:

In [250]:
scores_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6987 entries, 0 to 79
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   competition_id  6987 non-null   object 
 1   jumper_name     6987 non-null   object 
 2   jumper_country  6987 non-null   object 
 3   note_A          6984 non-null   float64
 4   note_B          6984 non-null   float64
 5   note_C          6984 non-null   float64
 6   note_D          6984 non-null   float64
 7   note_E          6984 non-null   float64
 8   referenced_A    6984 non-null   float64
 9   referenced_B    6984 non-null   float64
 10  referenced_C    6984 non-null   float64
 11  referenced_D    6984 non-null   float64
 12  referenced_E    6984 non-null   float64
dtypes: float64(10), object(3)
memory usage: 764.2+ KB


In [258]:
scores_df.describe()

,note_A,note_B,note_C,note_D,note_E,referenced_A,referenced_B,referenced_C,referenced_D,referenced_E
count,6984.000000,6984.000000,6984.000000,6984.000000,6984.000000,6984.000000,6984.000000,6984.000000,6984.000000,6984.000000
mean,17.750215,17.766180,17.725014,17.732030,17.775988,0.000412,0.020368,-0.031089,-0.022319,0.032628
std,1.146476,1.141918,1.151998,1.169309,1.592028,0.492640,0.477977,0.475486,0.484181,1.202295
min,9.500000,8.500000,8.000000,8.000000,8.000000,-9.250000,-9.250000,-9.000000,-8.875000,-2.250000
25%,17.500000,17.500000,17.000000,17.000000,17.500000,-0.250000,-0.250000,-0.250000,-0.250000,-0.250000
50%,18.000000,18.000000,18.000000,18.000000,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,18.500000,18.500000,18.500000,18.500000,18.500000,0.250000,0.250000,0.250000,0.250000,0.250000
max,20.000000,20.000000,20.000000,20.000000,53.500000,3.375000,3.375000,1.875000,2.500000,35.750000


In [259]:
scores_df.head(10)

,competition_id,jumper_name,jumper_country,note_A,note_B,note_C,note_D,note_E,referenced_A,referenced_B,referenced_C,referenced_D,referenced_E
0,2005JP3007RL.pdf,Ahonen Janne,FIN,19.5,18.5,19.0,19.0,19.0,0.625,-0.625,0.000,0.000,0.000
1,2005JP3007RL.pdf,Ahonen Janne,FIN,18.0,17.0,16.5,16.5,18.0,1.000,-0.250,-0.875,-0.875,1.000
2,2005JP3007RL.pdf,Malysz Adam,POL,19.0,19.0,18.0,18.5,18.5,0.500,0.500,-0.750,-0.125,-0.125
3,2005JP3007RL.pdf,Malysz Adam,POL,18.5,18.5,18.0,18.0,18.0,0.375,0.375,-0.250,-0.250,-0.250
4,2005JP3007RL.pdf,Janda Jakub,CZE,19.0,19.0,19.0,19.0,18.5,0.125,0.125,0.125,0.125,-0.500
5,2005JP3007RL.pdf,Janda Jakub,CZE,18.5,18.5,18.5,18.5,18.5,0.000,0.000,0.000,0.000,0.000
6,2005JP3007RL.pdf,Morgenstern Thomas,AUT,17.5,18.0,17.5,17.0,17.5,0.000,0.625,0.000,-0.625,0.000
7,2005JP3007RL.pdf,Morgenstern Thomas,AUT,19.5,19.0,18.5,19.0,18.5,0.750,0.125,-0.500,0.125,-0.500
8,2005JP3007RL.pdf,Hoellwarth Martin,AUT,18.0,19.0,17.5,18.5,18.0,-0.250,1.000,-0.875,0.375,-0.250
9,2005JP3007RL.pdf,Hoellwarth Martin,AUT,18.0,18.0,18.0,17.5,18.0,0.125,0.125,0.125,-0.500,0.125


As we see above, we got 6987 entries, in which 6984 non-null. That means we got three null entries that will be dropped. What's more, we can see a very generous note from judge E - 53.5, whilst in ski jumping maximum we can get is 20. The judges can give notes that are full or full and half points.

# Data cleaning

We will drop null entries and remove values that are above 20 or that are not x.0 or x.5 values.

In [24]:
scores_df_clean = scores_df.dropna()

for note_index in ['A', 'B', 'C', 'D', 'E']:
    column = f'note_{note_index}'
    scores_df_clean = scores_df_clean[
        (scores_df_clean[column]<=20) & (scores_df_clean[column]%0.5==0)
]

Check clean DataFrame

In [27]:
scores_df_clean.describe()

,note_A,note_B,note_C,note_D,note_E,referenced_A,referenced_B,referenced_C,referenced_D,referenced_E
count,6976.000000,6976.000000,6976.000000,6976.000000,6976.000000,6976.000000,6976.000000,6976.000000,6976.000000,6976.000000
mean,17.751433,17.767345,17.725917,17.732798,17.738389,0.010321,0.030211,-0.021574,-0.012973,-0.005985
std,1.145944,1.141489,1.151652,1.169593,1.137536,0.396322,0.379340,0.383509,0.396046,0.375815
min,9.500000,8.500000,8.000000,8.000000,8.000000,-2.625000,-3.125000,-5.500000,-6.375000,-2.250000
25%,17.500000,17.500000,17.000000,17.000000,17.500000,-0.250000,-0.250000,-0.250000,-0.250000,-0.250000
50%,18.000000,18.000000,18.000000,18.000000,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,18.500000,18.500000,18.500000,18.500000,18.500000,0.250000,0.250000,0.250000,0.250000,0.250000
max,20.000000,20.000000,20.000000,20.000000,20.000000,3.375000,3.375000,1.875000,2.500000,3.250000


# Creating a database

As this is an educational project we will create a database and insert our data into tables to get more skills in SQL. So some tasks will be performed directly using Pandas and some using SQL

## Create tables

In [34]:
db = sqlite3.connect('ski_jumping_database')
cursor = db.cursor()
cursor.execute('CREATE TABLE IF NOT EXISTS competitions (competition_id, year, referee_A, referee_B, referee_C, referee_D, referee_E)')
cursor.execute('CREATE TABLE IF NOT EXISTS jury (judge_name, judge_nationality)')
cursor.execute('CREATE TABLE IF NOT EXISTS notes (competition_id, jumper_name, jumper_country, note_A, note_B, note_C, note_D, note_E)')
db.commit()

## Insert data into tables

In [38]:
competition_data_df.to_sql('competitions', db, if_exists='replace', index=False)
jury_data_df.to_sql('jury', db, if_exists='replace', index=False)
scores_df_clean.to_sql('notes', db, if_exists='replace', index=False)

6976

Let's check if it works

In [58]:
cursor.execute("SELECT * FROM jury WHERE judge_nationality = 'POL';")
for row in cursor:
    print(row)

('Dlugopolski Kazimierz', 'POL')
('Szostak Tadeusz', 'POL')
('Siderek Marek', 'POL')
('Tucznio Marek', 'POL')
('Gunka Ryszard', 'POL')
('Nadarkiewicz Edward', 'POL')
('Pilch Jerzy', 'POL')
('Przybyta Edward', 'POL')


Seems ok

# Exploratory data analysis

At the beginning let's check how many judges and competitors of each nationality took part in our competitions. For training purposes we will get one value using SQL and one from Pandas.

In [71]:
# Create a DataFrame containing distinct values of jumpers and countries
jumpers_df = scores_df_clean.drop_duplicates(subset=['jumper_name', 'jumper_country'])[['jumper_name', 'jumper_country']]
jumpers_df.sort_values('jumper_country')

,jumper_name,jumper_country
78,Eigentler Markus,AUT
79,Hafele Mathias,AUT
77,Fettner Manuel,AUT
62,Koch Martin,AUT
79,Schneider Balthasar,AUT
...,...,...
44,Loitzl Wolfgang,WSC
58,Ritzerfeld Joerg,WSV
61,Mechler Maximilian,WSV
63,Hocke Stephan,WSV


We can see that on some cases there is jumper's team name instead of country ID. We will drop them.

In [82]:
jumpers_df = jumpers_df.loc[jumpers_df['jumper_country'].str.len()==3]
# Print values left
print(jumpers_df['jumper_country'].unique())

['FIN' 'POL' 'CZE' 'AUT' 'GER' 'JPN' 'NOR' 'SLO' 'FRA' 'SUI' 'RUS' 'BLR'
 'USA' 'SVK' 'EST' 'SWE' 'WSC' 'WSV' 'CSK' 'WKS' 'FIT' 'KAZ' 'CHN' 'KOR'
 'LKS' 'G.S' 'ITA' 'NED' 'BUL' 'KGZ']


In [86]:
# Drop remaining rows that are not country indexes
bad_index = ['WSC', 'WSV', 'CSK', 'WKS', 'FIT', 'LKS', 'G.S', 'KGZ']
jumpers_df = jumpers_df.loc[jumpers_df['jumper_country'].apply(lambda x: x not in bad_index)]
jumpers_df.head()

,jumper_name,jumper_country
0,Ahonen Janne,FIN
2,Malysz Adam,POL
4,Janda Jakub,CZE
6,Morgenstern Thomas,AUT
8,Hoellwarth Martin,AUT
